# A Random Network Model for Synaptic Plasticity

This notebook contains code accompanying the paper [A spatial small-world graph arising from activity-based reinforcement](https://arxiv.org/abs/1904.01817). 

We first implement the network dynamics and then rely on [TikZ](https://github.com/pgf-tikz/pgf) for visualization.

In [90]:

from IPython.display import HTML

# Youtube
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/A9zLKmt2nHo" frameborder="0" allowfullscreen></iframe>')


## The Model

We consider a conceptually simplified random network model for the phenomenon of [synaptic plasticity](https://en.wikipedia.org/wiki/Synaptic_plasticity). We model the synaptic weight by a stochastic process on a fixed graph $G = (V, E)$ thought of as a collection of neurons interconnected by synapses.

More precisely, consider the set of neurons $V = \mathbb Z \times \mathbb Z_{\ge 0}$, which we think of copies of $\mathbb Z$ organized in layers. The set of synapses $E$ is obtained by connecting each neuron $(k, h)$ in layer $h \ge 0$  to all neurons of the form  $(\ell, h + 1)$ with $|\ell - k| \le a^h$ for a model parameter $a >1$. Loosely speaking, $a$ governs the speed at which the visibility scope of a neuron increase, when moving to higher layers.

Additionally, the neurons feature iid heavy-tailed fitnesses $\{F_v\}_{v \in V}$ with a tail index $\gamma < 1$. 

On this marked network, we now define dynamically evolving synaptic weights $\{W_t(e)\}_{\substack{e \in E \\ t \ge 0 }}$ as follows. Each of the neurons is equipped with a Poisson clock. When the clock rings at neuron $v \in V$ at time $t \ge 0$, then the corresponding neuron fires and the weight of one of the incident synapses $(v, w)$ leading to the next layer increases by 1. The crux of the model lies in the selection mechanism. The selection of the synapse occurs at random with probability proportional to 
$$F_wW_{t−}(v, w)^\beta.$$

Hence, the model incorporates two effects. First, the fitter the neuron, the higher the probability that it is selected. Second, if a synapse was used frequently in the past, then it means that it is of importance and should be selected again with a higher probability in the future.

## Simulation of Network Dynamics

Now, we simulate the random network model as described above.

In [8]:
def simulate_network(a = 3,
                     beta = 1.5,
                     gamma = .2,
                     hrange = 250,
                     layers = 6, 
                     runs = 29):
    """Simulation of the network model
    # Arguments
        a: network parameter for scope increase
        beta: reinforcement strength for synaptic weights
        gamma: pareto parameter
        hrange: horizontal range of the network
        layers: number of layers
        runs: number of firings per neuron
       
    # Result
        fitnesses and evolution of edge weights
    """
    
    #initialize weights and fitnesses
    scope_sizes = [1 + 2 * a**layer for layer in range(layers)]     
    fits =  [np.random.rand(hrange) ** (-1 / gamma) for _ in range(layers)] 
    weights = [np.ones([hrange, scope]) for scope in scope_sizes]
    weight_trace = [weights]
    for _ in range(runs):
        #selection probabilities proportional to power weight and fitness
        weight_contrib = [weight_vec ** beta for weight_vec in weights]
        neighbs = neighb_list(a, hrange)
        fit_contribs = [[fit[neighb] for neighb in nb] for (fit, nb) in zip(fits, neighbs)]

        #compute selection probabilities
        sel_probs = [weight * fit for (weight, fit) in zip(weight_contrib, fit_contribs)]

        #select random edges 
        edge_choices = [np.apply_along_axis(lambda x: np.random.choice(len(x), 1, p = x / sum(x))[0], 
                                            1, 
                                            probs) 
               for probs in sel_probs]

        #one-hot encode selected edges
        increments = [one_hot(edge_choice, scope) for (edge_choice, scope) in zip(edge_choices, scope_sizes)]

        #update weights
        weights = [weight + increment for (weight, increment) in zip(weights, increments)]
        weight_trace += [weights]
    
    return (fits, weight_trace)

First, we need a function to represent the network as a list of neighbors. 

In [9]:
import numpy as np 
def neighb_list(a,
              hrange = 250,
              layers = 6):
    """Represent network structure as list of neighbors
    # Arguments
        a: network parameter for scope increase
        hrange: horizontal range of the network
        layers: number of layers
    # Result
        list of coordinates of neighbors
    """
    #centered_scopes
    c_scopes = [np.arange(-a ** layer, a**layer + 1) for layer in range(layers)]
    return([np.array([(idx + c_scope) % hrange for idx in range(hrange)]) for c_scope in c_scopes])

We need an auxiliary function to generate one-hot encoded vectors.

In [11]:
def one_hot(idxs, size):
    """Compute one-hot encoded vectors from an array of indexes
    # Arguments
        idxs: index array
        size: encoding dimension
    # Result
        one-hot encoded vectors as specified by indexes
    """
    one_hot = np.zeros([len(idxs), size])
    one_hot[range(len(idxs)), idxs] = 1
    return(one_hot)

In [12]:
#random seed
seed = 45
np.random.seed(seed)

fits, weight_trace = simulate_network()

## Visualization

Finally, we create a TikZ-visualization of the network dynamics.

In [34]:
def plot_synapses(idxs,
                  weights,
                  thresh = 1,
                  x_scale = .15,
                  node_scale = .2,
                  line_scale = 2,
                  gray_scale = 5,
                  max_line_width = 1.6):
    """Plot relevant synapses
    # Arguments
        idxs: indexes of layer-0 node
        weights: weights to be plotted
        thresh: weight-threshold for selecting a synapse
        x_scale: scaling in x-direction
        node_scale: scaling of nodes
        line_scale: scaling of line width
        gray_scale: gray_scale of lines
        max_line_width: maximum line width of graph
        
    # Result
        tikz representation of graph
    """
    
    
    layers = len(weights)
    hrange = len(weights[0])
    a = len(weights[0][0])
    
    result = []
    path = []
    neighbs = neighb_list(a, hrange)
    
    for layer in range(layers - 1):
        
        #collect only relevant edges
        rel_synaps = weights[layer] > thresh
        next_idxs = [neighbs[layer][idx][rel_synaps[idx]] for idx in idxs]
        rel_synaps = [rel_synaps[idx] for idx in idxs]
        
        #draw paths        
        string = "\\draw[line width = {5:1.2f}pt, black!{4:1.2f}!white]"
        string += " ({0:1.2f}, {1:1.2f})--({2:1.2f}, {3:1.2f});\n"
        path_unordered = [[string.format((idx % hrange) * x_scale,
                               layer,
                               (neighb % hrange) * x_scale,
                               layer + 1,
                               min(gray_scale * weight, 100),
                               min(line_scale * weight, max_line_width)),
                           weight]
                 for (idx, rel_synaps) in zip(idxs, rel_synaps)
                for (neighb, weight) in zip(neighbs[layer][idx][rel_synaps], 
                                           weights[layer][idx][rel_synaps])]
                                           
        #order for drawing
        ordering = np.argsort([el[1] for el in path_unordered])
        path += np.array([el[0] for el in path_unordered])[ordering].tolist()
        idxs = np.unique([y for x in next_idxs for y in x])
        
        #add point
        result += ["\\fill ({0:1.2f}, {1:1.2f}) circle ({2:1.1f}pt);\n".format((idx % hrange) * x_scale, 
                                                                          layer + 1,
                                                                          node_scale * np.log(fits)[layer, idx]) 
                   for idx in idxs]

    path += [';\n']
    return ''.join(path + result)

Draw paths from selected nodes

In [45]:
start_idx = 50
end_idx = 100

paths_trace = []

for i in range(5, len(weight_trace)):
    paths = plot_synapses( np.linspace(start_idx, 
                                         end_idx, 
                                         end_idx - start_idx + 1, 
                                         dtype = int), 
                          weight_trace[i])
    paths = '{\\begin{tikzpicture}\n' + paths + '\\end{tikzpicture}\n}'
    paths_trace += [paths]

Write to file

In [46]:
fname = 'coalesc.tex'

f = open(fname, "w")
for paths in paths_trace:
    f.write(paths)
f.close()

Compile pdf

In [47]:
!pdflatex evolFig.tex


This is pdfTeX, Version 3.14159265-2.6-1.40.19 (TeX Live 2019/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./evolFig.tex
LaTeX2e <2018-12-01>
(/usr/share/texlive/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(/usr/share/texlive/texmf-dist/tex/latex/tools/shellesc.sty)
(/usr/share/texlive/texmf-dist/tex/generic/oberdiek/ifluatex.sty)
(/usr/share/texlive/texmf-dist/tex/generic/oberdiek/ifpdf.sty)
(/usr/share/texlive/texmf-dist/tex/generic/ifxetex/ifxetex.sty)
(/usr/share/texlive/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/share/texlive/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/share/texlive/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/share/texlive/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/share/texlive/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: artic

Create GIF

In [48]:
!convert -density 150 -delay 16 -loop 1 -background white -alpha remove evolFig.pdf evolFig.gif